# Missing value analysis
In this notebook we analyze the surgical data set for missing values and how to deal with this. First, let's import the data set and see how many observations we have:

In [110]:
import pandas as pd
df = pd.read_csv('data/raw.csv')
len(df.index)

4086

## Overview
Let's see how many values are missing for each variable:

In [111]:
df.isnull().sum()

Operatietype                       119
Chirurg                              0
Anesthesioloog                     579
Benadering                         833
OK                                   0
Casustype                           36
Dagdeel                              0
Leeftijd                             1
Geslacht                           552
AF                                 554
Chronische longziekte              552
Extracardiale vaatpathie           552
Eerdere hartchirurgie              552
Actieve endocarditis               552
Kritische preoperatieve status     552
Myocard infact <90 dagen           552
Aorta chirurgie                    552
Pulmonale hypertensie              556
Linker ventrikel functie          3281
Euroscore1                         552
Euroscore2                        3129
Nierfunctie                       3291
Slechte mobiliteit                 552
BMI                               1180
DM                                 554
Hypercholesterolemie     

## Missing outcome variables
It stands out that `Operatieduur` is missing in two observations. Considering that we cannot train or test on these
rows, we delete them. We also remove the outcome variables for consistency.

In [112]:
df = df[df.Operatieduur.isnull() == False].drop(
    columns=['Operatieduur', 'Geplande operatieduur', 'Ziekenhuis ligduur', 'IC ligduur']
)
df.isnull().sum()

Operatietype                       118
Chirurg                              0
Anesthesioloog                     579
Benadering                         832
OK                                   0
Casustype                           36
Dagdeel                              0
Leeftijd                             1
Geslacht                           552
AF                                 554
Chronische longziekte              552
Extracardiale vaatpathie           552
Eerdere hartchirurgie              552
Actieve endocarditis               552
Kritische preoperatieve status     552
Myocard infact <90 dagen           552
Aorta chirurgie                    552
Pulmonale hypertensie              556
Linker ventrikel functie          3279
Euroscore1                         552
Euroscore2                        3127
Nierfunctie                       3289
Slechte mobiliteit                 552
BMI                               1179
DM                                 554
Hypercholesterolemie     

## Nearly empty observations
It stands out that a significant amount of predictors have approx. 552 missing values. We check how many of
those overlap, starting by checking observations were `Geslacht` is not present. 

In [113]:
df[df.Geslacht.isnull()].isnull().sum()

Operatietype                       35
Chirurg                             0
Anesthesioloog                    552
Benadering                        552
OK                                  0
Casustype                           9
Dagdeel                             0
Leeftijd                            0
Geslacht                          552
AF                                552
Chronische longziekte             552
Extracardiale vaatpathie          552
Eerdere hartchirurgie             552
Actieve endocarditis              552
Kritische preoperatieve status    552
Myocard infact <90 dagen          552
Aorta chirurgie                   552
Pulmonale hypertensie             552
Linker ventrikel functie          552
Euroscore1                        552
Euroscore2                        552
Nierfunctie                       552
Slechte mobiliteit                552
BMI                               552
DM                                552
Hypercholesterolemie              552
Hypertensie 

It appears that if `Geslacht` is missing, 26 out of 32 predictors will certainly be missing. These 552 observations thereby cause the following percentage of missing values:

In [114]:
round(df[df.Geslacht.isnull()].isnull().sum().sum() / df.isnull().sum().sum() * 100, 2)

57.1

Outside of these 552 observations, the following missing values remain:

In [115]:
df[df.Geslacht.notnull()].isnull().sum()

Operatietype                        83
Chirurg                              0
Anesthesioloog                      27
Benadering                         280
OK                                   0
Casustype                           27
Dagdeel                              0
Leeftijd                             1
Geslacht                             0
AF                                   2
Chronische longziekte                0
Extracardiale vaatpathie             0
Eerdere hartchirurgie                0
Actieve endocarditis                 0
Kritische preoperatieve status       0
Myocard infact <90 dagen             0
Aorta chirurgie                      0
Pulmonale hypertensie                4
Linker ventrikel functie          2727
Euroscore1                           0
Euroscore2                        2575
Nierfunctie                       2737
Slechte mobiliteit                   0
BMI                                627
DM                                   2
Hypercholesterolemie     

## Nearly empty predictors
The fraction of columns that are missing:

In [116]:
df_temp = df.isnull().sum() / len(df)
df_temp

Operatietype                      0.028893
Chirurg                           0.000000
Anesthesioloog                    0.141773
Benadering                        0.203722
OK                                0.000000
Casustype                         0.008815
Dagdeel                           0.000000
Leeftijd                          0.000245
Geslacht                          0.135162
AF                                0.135651
Chronische longziekte             0.135162
Extracardiale vaatpathie          0.135162
Eerdere hartchirurgie             0.135162
Actieve endocarditis              0.135162
Kritische preoperatieve status    0.135162
Myocard infact <90 dagen          0.135162
Aorta chirurgie                   0.135162
Pulmonale hypertensie             0.136141
Linker ventrikel functie          0.802889
Euroscore1                        0.135162
Euroscore2                        0.765671
Nierfunctie                       0.805338
Slechte mobiliteit                0.135162
BMI        

It stands out that there are three columns that miss over 50% of their data:

In [117]:
df_temp[df_temp > 0.5]

Linker ventrikel functie    0.802889
Euroscore2                  0.765671
Nierfunctie                 0.805338
dtype: float64

Which cause the following percentage of missing values:

In [118]:
df_temp = df.isnull().sum()
round(df_temp[df_temp > len(df.index) / 2].sum() / df.isnull().sum().sum() * 100, 2)

38.46

## Near empty observations and predictors
Together, the near empty observations and predictors mentioned above are responsible for the following percentage of missing values:

In [119]:
observations_missing = df[df.Geslacht.isnull()].isnull().sum().sum()
df_temp = df[df.Geslacht.notnull()].isnull().sum()
predictors_missing = df_temp[df_temp > (len(df.index) / 2) - 552].sum()
round((observations_missing + predictors_missing) / df.isnull().sum().sum() * 100, 2)


88.99

## Missing continuous predictors
Where discrete predictors can be assigned an unknown class, this is not possible for predictors variables. Out of the 32 predictors, the following 5 are continuous:

* Leeftijd
* Euroscore 1
* Euroscore 2
* BMI
* Aantal anastomosen

### Leeftijd

In [120]:
df.Leeftijd.isnull().sum()

1

In [121]:
df[df.Leeftijd.isnull()].isnull().sum()

Operatietype                      0
Chirurg                           0
Anesthesioloog                    0
Benadering                        0
OK                                0
Casustype                         0
Dagdeel                           0
Leeftijd                          1
Geslacht                          0
AF                                0
Chronische longziekte             0
Extracardiale vaatpathie          0
Eerdere hartchirurgie             0
Actieve endocarditis              0
Kritische preoperatieve status    0
Myocard infact <90 dagen          0
Aorta chirurgie                   0
Pulmonale hypertensie             0
Linker ventrikel functie          0
Euroscore1                        0
Euroscore2                        0
Nierfunctie                       0
Slechte mobiliteit                0
BMI                               1
DM                                0
Hypercholesterolemie              0
Hypertensie                       0
Perifeer vaatlijden         

### Euroscore 1

In [122]:
df.Euroscore1.isnull().sum()

552

In [123]:
df[df.Euroscore1.isnull()].isnull().sum()

Operatietype                       35
Chirurg                             0
Anesthesioloog                    552
Benadering                        552
OK                                  0
Casustype                           9
Dagdeel                             0
Leeftijd                            0
Geslacht                          552
AF                                552
Chronische longziekte             552
Extracardiale vaatpathie          552
Eerdere hartchirurgie             552
Actieve endocarditis              552
Kritische preoperatieve status    552
Myocard infact <90 dagen          552
Aorta chirurgie                   552
Pulmonale hypertensie             552
Linker ventrikel functie          552
Euroscore1                        552
Euroscore2                        552
Nierfunctie                       552
Slechte mobiliteit                552
BMI                               552
DM                                552
Hypercholesterolemie              552
Hypertensie 

### Euroscore 2

In [124]:
df.Euroscore2.isnull().sum()

3127

In [125]:
df[df.Euroscore2.isnull()].isnull().sum()

Operatietype                       114
Chirurg                              0
Anesthesioloog                     576
Benadering                         772
OK                                   0
Casustype                           30
Dagdeel                              0
Leeftijd                             0
Geslacht                           552
AF                                 552
Chronische longziekte              552
Extracardiale vaatpathie           552
Eerdere hartchirurgie              552
Actieve endocarditis               552
Kritische preoperatieve status     552
Myocard infact <90 dagen           552
Aorta chirurgie                    552
Pulmonale hypertensie              552
Linker ventrikel functie          3047
Euroscore1                         552
Euroscore2                        3127
Nierfunctie                       3048
Slechte mobiliteit                 552
BMI                               1024
DM                                 552
Hypercholesterolemie     

### BMI

In [126]:
df.BMI.isnull().sum()

1179

In [127]:
df[df.BMI.isnull()].isnull().sum()

Operatietype                        50
Chirurg                              0
Anesthesioloog                     566
Benadering                         750
OK                                   0
Casustype                           17
Dagdeel                              0
Leeftijd                             1
Geslacht                           552
AF                                 552
Chronische longziekte              552
Extracardiale vaatpathie           552
Eerdere hartchirurgie              552
Actieve endocarditis               552
Kritische preoperatieve status     552
Myocard infact <90 dagen           552
Aorta chirurgie                    552
Pulmonale hypertensie              552
Linker ventrikel functie          1043
Euroscore1                         552
Euroscore2                        1024
Nierfunctie                       1043
Slechte mobiliteit                 552
BMI                               1179
DM                                 552
Hypercholesterolemie     

### Aantal anastomosen

In [128]:
df['Aantal anastomosen'].isnull().sum()

554

In [129]:
df[df['Aantal anastomosen'].isnull()].isnull().sum()

Operatietype                       35
Chirurg                             0
Anesthesioloog                    552
Benadering                        553
OK                                  0
Casustype                           9
Dagdeel                             0
Leeftijd                            0
Geslacht                          552
AF                                552
Chronische longziekte             552
Extracardiale vaatpathie          552
Eerdere hartchirurgie             552
Actieve endocarditis              552
Kritische preoperatieve status    552
Myocard infact <90 dagen          552
Aorta chirurgie                   552
Pulmonale hypertensie             553
Linker ventrikel functie          553
Euroscore1                        552
Euroscore2                        552
Nierfunctie                       554
Slechte mobiliteit                552
BMI                               552
DM                                552
Hypercholesterolemie              552
Hypertensie 